<a href="https://colab.research.google.com/github/jeremy-feng/deep-learning-coursework/blob/main/2-BERT/BERT-QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作业2：对Bert进行微调，完成QA任务

**如果你对Bert没有了解，请先观看视频 [BERT 论文逐段精读【论文精读】](https://www.bilibili.com/video/BV1PL411M7eQ)**

注：本次作业并不需要预先了解任何Transformer的知识，如有兴趣，可以在观看Bert的视频前，先预习 [Transformer论文逐段精读【论文精读】](https://www.bilibili.com/video/BV1pu411o7BE)，后续课程中会讲解Transformer的知识。



In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pwd

/content


In [9]:
import pandas as pd
import json
from tqdm import tqdm
import torch
import numpy as np
import random

device = "cuda" if torch.cuda.is_available() else "cpu"


# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(0)

In [10]:
with open('/content/drive/MyDrive/Colab Notebooks/deep-learning/2-BERT/cmrc2018/train.json') as f:
    train = json.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/deep-learning/2-BERT/cmrc2018/dev.json') as f:
    dev = json.load(f)


Let's have a glance at the data.

In [11]:
!pip install transformers
from transformers import BertTokenizerFast, BertForQuestionAnswering

# You can explore more pretrained models from https://huggingface.co/models
# 这段代码利用Hugging Face库中的BertTokenizerFast方法从预训练模型'bert-base-chinese'中加载tokenizer。
# 这个预训练模型是一个中文BERT模型，可以将中文句子或文本数据转换为相应的token，以便进行文本分类、序列标注等自然语言处理任务。
# BertTokenizerFast是BertTokenizer的升级版，速度更快，性能更优。
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese').to(device)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

## PreProcessing

### Prepare training data

In [12]:
paragraphs = []
questions = []
start_positions = []
end_positions = []
for paragraph in train['data']:
    for qa in paragraph['paragraphs'][0]['qas']:
        
        ### START CODE HERE ### 
        # For each question, add its paragraph, question, start_position and end_position(after calculation) to its corresponding list.
        paragraphs.append(paragraph['paragraphs'][0]['context'])
        questions.append(qa['question'])
        start_position = qa['answers'][0]['answer_start']
        start_positions.append(start_position)
        anwser_length = len(qa['answers'][0]['text'])
        end_positions.append(start_position + anwser_length)
        ### END CODE HERE ###

In [13]:
questions[:5]

['范廷颂是什么时候被任为主教的？',
 '1990年，范廷颂担任什么职务？',
 '范廷颂是于何时何地出生的？',
 '1994年3月，范廷颂担任什么职务？',
 '范廷颂是何时去世的？']

In [14]:
start_positions[:5]

[30, 41, 97, 548, 759]

In [15]:
end_positions[:5]

[35, 62, 126, 598, 780]

查看第 3 个问题的回答是否正确

In [16]:
paragraphs[0][start_positions[2]: end_positions[2]]

'范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'

将 paragraphs 和 questions 进行 encoding。
参考：[https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__)

In [17]:
# 下面这段代码使用了 Hugging Face 的 tokenizer 方法，将 paragraphs 和 questions 转换成相应的 token，
# 返回一个字典 (train_encodings) 包含这些 token 的各种信息，这些信息包括 input_ids、attention_mask 等等。
# return_tensors='pt' 表示返回 PyTorch 下的 tensor 格式
# padding 用于填充不足 max_length 的 token
# truncation 用于在超过 max_length 时截断 token
# 最终的 token 长度被限制在 512 内
train_encodings = tokenizer(
    paragraphs,
    questions,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
)


In [36]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

- 在问答任务中，`input_ids` 是将输入文本转换为整数序列后的输出。它将每个单词或子词映射到一个唯一的整数 ID, 位于 [CLS] 和 [SEP] 标记会被分别映射到一个特殊的 ID，(101: CLS, 102: SEP)。具体可以参考下方例子。

- 在 `token_type_ids` 中，这些标记的值通常为 0 或 1，其中 0 表示该 token 属于第一个文本序列（通常是问题），1 表示该 token 属于第二个文本序列（通常是段落）。

- 在 `attention_mask` 中，0 表示对应的标记应该被忽略，1 表示对应的标记应该被关注。当输入序列长度不足最大长度时，我们需要在序列末尾填充一些无意义的标记，以使序列长度达到最大长度。在这种情况下，`tokenizer`将填充的标记的 attention mask 设置为 0，以告诉模型它们不应该被关注。

In [19]:
a = tokenizer(
    paragraphs[0],
    questions[0],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
)['input_ids'][0]

In [20]:
b = tokenizer(
    paragraphs[0],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
)['input_ids'][0]

In [21]:
tokenizer.decode([711, 1921,  712, 3136, 3777, 1079, 2600, 3136, 1277,
        2134, 2429, 5392,  102, 5745, 2455, 7563, 3221,  784,  720, 3198,  952,
        6158,  818,  711,  712, 3136, 4638, 8043,  102])

'为 天 主 教 河 内 总 教 区 宗 座 署 [SEP] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP]'

In [22]:
len([5745, 2455, 7563, 3221,  784,  720, 3198,  952,
        6158,  818,  711,  712, 3136, 4638, 8043,  102])

16

最后 16 个元素为 1，这说明最后 16 个 token 对应的是 question

In [23]:
train_encodings['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
train_encodings['token_type_ids'][0].sum()

tensor(16)

In [25]:
tokenizer.decode([711, 1921,  712, 3136, 3777, 1079, 2600, 3136, 1277,
        2134, 2429, 5392, 4415,  809, 1856, 6133, 6421, 3136, 1277, 2600,  712,
        3136, 4638, 4958, 5375,  511, 8447, 2399,  102])

'为 天 主 教 河 内 总 教 区 宗 座 署 理 以 填 补 该 教 区 总 主 教 的 空 缺 。 1994 年 [SEP]'

In [26]:
train_encodings['token_type_ids']

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [27]:
print(train_encodings['input_ids'].shape)
print(train_encodings['token_type_ids'].shape)
print(train_encodings['attention_mask'].shape)

torch.Size([10142, 512])
torch.Size([10142, 512])
torch.Size([10142, 512])


下面的代码的作用是：将 answer 在原始 paragrapgh 的起止索引，转换为在经过tokenizor 之后点 input_ids 中的起止索引

In [28]:
# `char_to_token` will convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
train_encodings['start_positions'] = torch.tensor([train_encodings.char_to_token(idx, x) if train_encodings.char_to_token(idx, x) != None else -1
                                      for idx, x in enumerate(start_positions)])
train_encodings['end_positions'] = torch.tensor([train_encodings.char_to_token(idx, x-1) if train_encodings.char_to_token(idx, x-1) != None else -1
                                    for idx, x in enumerate(end_positions)])

In [29]:
train_encodings['start_positions']

tensor([ 31,  39,  86,  ..., 142, 225,  17])

In [30]:
train_encodings['end_positions']

tensor([ 32,  56, 110,  ..., 143, 244,  19])

以第 3 个问题为例，即以 86 和 110 作为起止索引为例，查看它们对应的字符串是什么

In [31]:
questions[2]

'范廷颂是于何时何地出生的？'

In [32]:
paragraphs[0][start_positions[2]: end_positions[2]]

'范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'

In [35]:
tokenizer.decode(train_encodings['input_ids'][0][86: 110+1])

'范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生'

### Prepare Dataset

In [37]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
    
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {k: v[idx].to(device) for k, v in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)

Automatic Mixed Precision (AMP) is available on NVIDIA GPUs that support Tensor Cores, which are specialized hardware units for performing fast matrix multiplication and convolution operations in deep learning. Specifically, Tensor Cores are available on NVIDIA Volta, Turing, and Ampere architectures, which include the following GPU series:

- Volta: Tesla V100, Titan V
- Turing: Quadro RTX, GeForce RTX 20-series, Titan RTX
- Ampere: A100, GeForce RTX 30-series, Titan RTX

In [38]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)
fp16_training = True

if fp16_training:
    !pip install accelerate
    from accelerate import Accelerator

    accelerator = Accelerator()
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.3 MB/s eta 0:00:00


In [39]:
device

device(type='cuda')

In [48]:
next(model.parameters())

Parameter containing:
tensor([[ 0.0262,  0.0109, -0.0187,  ...,  0.0903,  0.0028,  0.0064],
        [ 0.0021,  0.0216,  0.0011,  ...,  0.0809,  0.0018,  0.0249],
        [ 0.0147,  0.0005,  0.0028,  ...,  0.0836,  0.0121,  0.0282],
        ...,
        [ 0.0346,  0.0021,  0.0085,  ...,  0.0085,  0.0337,  0.0099],
        [ 0.0541,  0.0289,  0.0263,  ...,  0.0526,  0.0651,  0.0353],
        [ 0.0200,  0.0023, -0.0089,  ...,  0.0799, -0.0562,  0.0247]],
       device='cuda:0', requires_grad=True)

In [47]:
next(model.parameters()).shape

torch.Size([21128, 768])

In [49]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

### START CODE HERE ### 
# Use AdamW as the optimizer, and learning rate 5e-5.
# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)
### END CODE HERE ### 


In [52]:
model, optim, train_loader = accelerator.prepare(model, optim, train_loader)

In [53]:
model.train()
loss_sum = 0.0
acc_start_sum = 0.0
acc_end_sum = 0.0

In [62]:
for batch_idx, batch in enumerate(tqdm(train_loader)):
    print(batch_idx, batch)
    break

  0%|          | 0/1268 [00:00<?, ?it/s]

0 {'input_ids': tensor([[ 101, 7770, 7965,  ...,  720, 8043,  102],
        [ 101,  730, 3780,  ...,    0,    0,    0],
        [ 101,  517, 1520,  ...,    0,    0,    0],
        ...,
        [ 101, 2476, 5739,  ...,    0,    0,    0],
        [ 101,  517, 2769,  ...,    0,    0,    0],
        [ 101, 1765, 1898,  ...,    0,    0,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'start_positions': tensor([ 14,  57, 136,  43, 120, 307,  53, 204], device='cuda:0'), 'end_positions': tensor([ 20,  59, 137,  52, 1

In [63]:
batch_idx

0

由于 train_loader 设置了 batch_size=8，因此这里每一个批次包含 8 个数据



In [64]:
batch

{'input_ids': tensor([[ 101, 7770, 7965,  ...,  720, 8043,  102],
         [ 101,  730, 3780,  ...,    0,    0,    0],
         [ 101,  517, 1520,  ...,    0,    0,    0],
         ...,
         [ 101, 2476, 5739,  ...,    0,    0,    0],
         [ 101,  517, 2769,  ...,    0,    0,    0],
         [ 101, 1765, 1898,  ...,    0,    0,    0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 'start_positions': tensor([ 14,  57, 136,  43, 120, 307,  53, 204], device='cuda:0'),
 'end_positions': tensor([ 

In [65]:
optim.zero_grad()

In [66]:
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']
start_positions = batch['start_positions']
end_positions = batch['end_positions']

outputs = model(input_ids, attention_mask=attention_mask, 
                start_positions=start_positions, 
                end_positions=end_positions)

In [67]:
outputs

QuestionAnsweringModelOutput(loss=tensor(6.4114, device='cuda:0', grad_fn=<DivBackward0>), start_logits=tensor([[ 0.1751, -0.1216,  0.4636,  ...,  0.5935,  0.4839,  0.7081],
        [-0.1473, -0.3509,  0.1302,  ..., -0.2002, -0.3946, -0.3744],
        [ 0.2786,  0.0641,  0.4189,  ...,  0.1372,  0.2105,  0.1368],
        ...,
        [-0.0034,  0.1997,  0.1095,  ...,  0.2093,  0.1921,  0.0117],
        [ 0.5537, -0.0444,  0.5240,  ...,  0.3153,  0.1386,  0.0803],
        [ 0.0620,  0.0415,  0.1780,  ...,  0.0031, -0.2416, -0.1601]],
       device='cuda:0', grad_fn=<CloneBackward0>), end_logits=tensor([[ 0.8021,  0.6094, -0.2039,  ...,  0.5187,  0.6640,  0.2209],
        [ 0.1915,  0.7896, -0.2102,  ...,  0.0642,  0.2232,  0.0899],
        [ 0.3394,  0.5749,  0.5838,  ..., -0.1420, -0.1652, -0.0355],
        ...,
        [ 0.4693,  0.5062, -0.0599,  ..., -0.0544, -0.2077, -0.1598],
        [ 0.7608,  0.4795, -0.1417,  ..., -0.0799, -0.2762,  0.1039],
        [ 0.8589,  0.9004, -0.2578,  

In [68]:
loss = outputs.loss

In [69]:
loss

tensor(6.4114, device='cuda:0', grad_fn=<DivBackward0>)

In [70]:
accelerator.backward(loss)

In [72]:
optim.step()

In [73]:
loss_sum += loss.item()

对于第一个问答，输出每个 token 作为 start 的 logit，其值越大，说明越有可能作为 start

In [83]:
outputs.start_logits[0]

tensor([ 1.7514e-01, -1.2164e-01,  4.6360e-01,  3.0065e-01,  2.9359e-01,
         5.4023e-01,  3.7192e-01,  2.0874e-01,  1.7231e-01,  7.1841e-01,
         1.3333e-01,  5.4579e-01,  5.5089e-01,  5.3872e-01,  4.7586e-01,
         7.1148e-01,  6.5530e-01,  8.3357e-01,  6.0856e-01,  5.7281e-01,
         1.1920e-01, -1.5845e-01,  6.6009e-02, -2.3724e-02, -1.2359e-01,
        -2.5370e-01, -2.3190e-01,  3.7006e-01,  4.1520e-01,  2.9374e-01,
         3.1413e-01, -8.0037e-02,  1.8278e-03,  7.2287e-01,  1.0099e-01,
         2.3198e-01,  9.7436e-02,  5.8377e-01,  2.3440e-01,  8.6625e-01,
         6.2343e-01,  3.7204e-01,  3.1254e-01,  1.0517e+00,  1.2295e-01,
         9.6936e-02,  8.8488e-01,  2.4407e-01, -3.6297e-01, -3.4003e-02,
         7.0640e-02,  2.9501e-01, -5.1674e-01,  6.4323e-01, -1.1721e-01,
         5.3592e-01,  2.5208e-01,  2.3752e-01, -2.9283e-01,  4.7447e-02,
        -3.3944e-02,  2.2520e-01, -1.1115e-01,  1.9890e-01,  3.2585e-01,
         5.5852e-02,  5.1859e-01,  3.4758e-04, -8.0

以下结果说明，第 1 个问答中，第 306 个 token 最有可能是 start

In [88]:
torch.argmax(outputs.start_logits, dim=1)

tensor([306,  35, 298,  20, 283,  62,  22, 285], device='cuda:0')

In [89]:
start_pred = torch.argmax(outputs.start_logits, dim=1)
end_pred = torch.argmax(outputs.end_logits, dim=1)

In [90]:
start_pred

tensor([306,  35, 298,  20, 283,  62,  22, 285], device='cuda:0')

In [91]:
end_pred

tensor([143,   1, 234, 313, 198, 359, 196,   1], device='cuda:0')

In [ ]:
if fp16_training:
    model, optim, train_loader = accelerator.prepare(model, optim, train_loader)
    
model.train()
for epoch in range(3):
    loss_sum = 0.0
    acc_start_sum = 0.0
    acc_end_sum = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for batch_idx, batch in enumerate(pbar):
        optim.zero_grad()
        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']
        
        outputs = model(input_ids, attention_mask=attention_mask, 
                        start_positions=start_positions, 
                        end_positions=end_positions)
        
        loss = outputs.loss
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optim.step()
        
        loss_sum += loss.item()
        
        ### START CODE HERE ### 
        # Obtain answer by choosing the most probable start position / end position
        # Using `torch.argmax` and its `dim` parameter to extract preditions for start position and end position.
        start_pred = torch.argmax(outputs.start_logits, dim=1)
        end_pred = torch.argmax(outputs.end_logits, dim=1)
        
        # calculate accuracy for start and end positions. eg., using start_pred and start_positions to calculate acc_start.
        acc_start = (start_pred == start_positions).float().mean()
        acc_end = (end_pred == end_positions).float().mean()
        ### END CODE HERE ### 
        
        acc_start_sum += acc_start
        acc_end_sum += acc_end
        
        # Update progress bar
        postfix = {
            "loss": f"{loss_sum/(batch_idx+1):.4f}",
            "acc_start": f"{acc_start_sum/(batch_idx+1):.4f}",
            "acc_end": f"{acc_end_sum/(batch_idx+1):.4f}"
        }

        # Add batch accuracy to progress bar
        batch_desc = f"Epoch {epoch}, train loss: {postfix['loss']}"
        pbar.set_postfix_str(f"{batch_desc}, acc start: {postfix['acc_start']}, acc end: {postfix['acc_end']}")


Epoch 0:  50%|█████     | 639/1268 [08:59<08:53,  1.18it/s, Epoch 0, train loss: 2.4209, acc start: 0.3885, acc end: 0.3783]

In [ ]:
def predcit(doc, query):
    print(doc)
    print('提问：', query)
    item = tokenizer([doc, query], max_length=512, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        input_ids = item['input_ids'].to(device).reshape(1,-1)
        attention_mask = item['attention_mask'].to(device).reshape(1,-1)
        
        outputs = model(input_ids[:, :512], attention_mask[:, :512])
        
        ### START CODE HERE ### 
        # Using `torch.argmax` and its `dim` parameter to extract preditions for start position and end position.
        start_pred = 
        end_pred = 
        ### END CODE HERE ### 
    
    try:
        start_pred = item.token_to_chars(0, start_pred)
        end_pred = item.token_to_chars(0, end_pred)
    except:
        return ''
    
    if start_pred.start > end_pred.end:
        return ''
    else:
        return doc[start_pred.start:end_pred.end]

In [ ]:
dev['data'][100]

{'paragraphs': [{'id': 'DEV_109',
   'context': '岑朗天（），笔名朗天、霍惊觉。香港作家、影评人、文化活动策划、大学兼职讲师。香港新亚研究所硕士，师从牟宗三，父亲为香港专栏作家昆南。曾在香港多家报社从事繙译、编辑、采访工作。1995年加入香港电影评论学会，并于2003-2007年出任该会会长，2016年退出。1995年参与创立新研哲学会，后易名香港人文哲学会，再易名香港人文学会。1998年加入树宁．现在式单位，出任该剧团董事及编剧。2003年担任牛棚书展（2003-6）统筹，协助开拓主流以外的书展文化（牛棚书展精神后为九龙城书节继承）。2004年6月至2011年加入商业电台光明顶，担任嘉宾主持。2004年至2014年于香港中文大学新闻与传播学院兼职教授媒体创意写作。2012年始兼任香港浸会大学电影学院讲师，教授文学与影视相关课程。',
   'qas': [{'question': '岑朗天笔名叫什么？',
     'id': 'DEV_109_QUERY_0',
     'answers': [{'text': '朗天、霍惊觉', 'answer_start': 8},
      {'text': '朗天、霍惊觉', 'answer_start': 8},
      {'text': '朗天、霍惊觉', 'answer_start': 8}]},
    {'question': '岑朗天的职业都有哪些？',
     'id': 'DEV_109_QUERY_1',
     'answers': [{'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17},
      {'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17},
      {'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17}]},
    {'question': '岑朗天哪年加入香港电影评论学会？',
     'id': 'DEV_109_QUERY_2',
     'answers': [{'text': '1995年', 'answer_start': 87},
      {'te

In [ ]:
model.eval()
predcit(dev['data'][100]['paragraphs'][0]['context'],
       dev['data'][100]['paragraphs'][0]['qas'][0]['question'])

岑朗天（），笔名朗天、霍惊觉。香港作家、影评人、文化活动策划、大学兼职讲师。香港新亚研究所硕士，师从牟宗三，父亲为香港专栏作家昆南。曾在香港多家报社从事繙译、编辑、采访工作。1995年加入香港电影评论学会，并于2003-2007年出任该会会长，2016年退出。1995年参与创立新研哲学会，后易名香港人文哲学会，再易名香港人文学会。1998年加入树宁．现在式单位，出任该剧团董事及编剧。2003年担任牛棚书展（2003-6）统筹，协助开拓主流以外的书展文化（牛棚书展精神后为九龙城书节继承）。2004年6月至2011年加入商业电台光明顶，担任嘉宾主持。2004年至2014年于香港中文大学新闻与传播学院兼职教授媒体创意写作。2012年始兼任香港浸会大学电影学院讲师，教授文学与影视相关课程。
提问： 岑朗天笔名叫什么？


'朗天、霍惊觉'

## Open Questions
可以查阅相关资料，并完成如下开放式的问答题。


- 我们使用了512长度的Bert，但是在实际应用中，输入长度可能大于512，你想怎么解决这个问题，请描述你的算法，在训练和预测时分别采取什么样的方法。（假设问题的长度都满足小于512token，段落的长度可能大于512token，以QA问题为例）


Your Answer:

- 在输出中，我们分别对start_pred和end_pred的位置进行预估，如果end_pred<start_pred，我们可以如何解决这样的问题?

Your Answer:

- Bert的分词方式是什么?在中文中，你觉得这样的方式会带来什么问题？什么样的分词方式适合中文？在中文的文本上，除了改变分词方式，还有哪些方式可以提升模型效果？

阅读资料：https://github.com/ymcui/Chinese-BERT-wwm

Your Answer: